In [19]:
# Import Dependencies
import pandas as pd

In [20]:
# input file names
movies_csv = "movies_metadata.csv"
credits_csv = "credits.csv"

In [21]:
# start with credits file and extract director name
credits_df = pd.read_csv(credits_csv,low_memory=False)
credits_df.count()

cast    45476
crew    45476
id      45476
dtype: int64

In [22]:
movie_id = []
director = []

# loop thru each row 
for idx, rows in credits_df.iterrows():

# crew is stored as string, evaluate it to a list of dictionaries    
    crew_dict = eval(rows['crew'])
# look thru each dictionary and extract director name based on job title
    for i in range(len(crew_dict)):
        try:
            if crew_dict[i]['job'].lower() == 'director':
                director.append(crew_dict[i]['name'])
                movie_id.append(rows.id)
                break
        except:
            director.append("n/a")
            movie_id.append(rows.id)
            break 

# create Director dataframe to merge with movies data
movie_director_df = pd.DataFrame({"id": movie_id, "Director": director})
movie_director_df.count()


id          44589
Director    44589
dtype: int64

In [23]:
# read movies file
movies_df = pd.read_csv(movies_csv,low_memory=False)
movies_dataprep = movies_df[['genres', 'budget', 'id', 'title', 'popularity', 
                             'release_date', 'revenue']].copy()
# remove rows with junk movie ids
movies_dataprep = movies_dataprep[movies_dataprep.id != '8/20/1997']
movies_dataprep = movies_dataprep[movies_dataprep.id != '9/29/2012']
movies_dataprep = movies_dataprep[movies_dataprep.id != '1/1/2014']

# typecast to int for merging
movies_dataprep.id = movies_dataprep.id.astype('int')
movie_director_df.id = movie_director_df.id.astype('int')

In [24]:
movies_dataprep.count()

genres          45463
budget          45463
id              45463
title           45460
popularity      45460
release_date    45376
revenue         45460
dtype: int64

In [25]:
# merge movies df to director df on movie id
movies_with_director = pd.merge(movies_dataprep, movie_director_df, on = "id", how="inner")
movies_with_director.head()

,genres,budget,id,title,popularity,release_date,revenue,Director
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,862,Toy Story,21.946943,10/30/1995,373554033.0,John Lasseter
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,8844,Jumanji,17.015539,12/15/1995,262797249.0,Joe Johnston
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,Grumpier Old Men,11.7129,12/22/1995,0.0,Howard Deutch
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,31357,Waiting to Exhale,3.859495,12/22/1995,81452156.0,Forest Whitaker
4,"[{'id': 35, 'name': 'Comedy'}]",0,11862,Father of the Bride Part II,8.387519,2/10/1995,76578911.0,Charles Shyer


In [26]:
movies_with_director.count()

genres          44651
budget          44651
id              44651
title           44648
popularity      44648
release_date    44616
revenue         44648
Director        44651
dtype: int64

In [27]:
# convert to numeric for calculation
movies_with_director.budget = pd.to_numeric(movies_with_director.budget, errors='coerce')
movies_with_director.revenue = pd.to_numeric(movies_with_director.revenue, errors='coerce')

In [12]:
#filter for budget >= 5000 and revenue >= 5000
movies_with_director = movies_with_director[(movies_with_director.budget >= 5000)]
movies_with_director = movies_with_director[(movies_with_director.revenue >= 5000)]

In [28]:
movies_with_director.count()

genres          44651
budget          44651
id              44651
title           44648
popularity      44648
release_date    44616
revenue         44648
Director        44651
dtype: int64

In [29]:
# add ROI column
movies_with_director['ROI'] = 100 * ((movies_with_director.revenue - movies_with_director.budget) 
                                     / movies_with_director.budget)
movies_with_director.head()

,genres,budget,id,title,popularity,release_date,revenue,Director,ROI
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,862,Toy Story,21.946943,10/30/1995,373554033.0,John Lasseter,1145.180110
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,8844,Jumanji,17.015539,12/15/1995,262797249.0,Joe Johnston,304.303460
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,Grumpier Old Men,11.7129,12/22/1995,0.0,Howard Deutch,NaN
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,31357,Waiting to Exhale,3.859495,12/22/1995,81452156.0,Forest Whitaker,409.075975
4,"[{'id': 35, 'name': 'Comedy'}]",0,11862,Father of the Bride Part II,8.387519,2/10/1995,76578911.0,Charles Shyer,inf


In [18]:
movies_with_director.to_csv('movies_with_director.csv', index=False)

In [30]:
budget = []
genres = []
movie_id = []
title = []
popularity = []
release_date = []
revenue = []
ROI = []
director = []

for idx, rows in movies_with_director.iterrows():
    genres_dict = eval(rows['genres'])
    for i in range(len(genres_dict)):
        budget.append(rows.budget)
        movie_id.append(rows.id)
        title.append(rows.title)
        popularity.append(rows.popularity)
        release_date.append(rows.release_date)
        revenue.append(rows.revenue)
        ROI.append(rows.ROI)
        director.append(rows.Director)
        try:
            genres.append(genres_dict[i]['name'])
        except:
            genres.append("n/a")
            break
    
movies_final = pd.DataFrame({"genres": genres, "movie_id": movie_id, "budget": budget, 
                                     "title": title, "popularity": popularity, 
                                     "release_date": release_date, "revenue":revenue, 
                                     "ROI":ROI, "Director": director})
movies_final.head()

,genres,movie_id,budget,title,popularity,release_date,revenue,ROI,Director
0,Animation,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
1,Comedy,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
2,Family,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
3,Adventure,8844,65000000,Jumanji,17.015539,12/15/1995,262797249.0,304.30346,Joe Johnston
4,Fantasy,8844,65000000,Jumanji,17.015539,12/15/1995,262797249.0,304.30346,Joe Johnston


In [31]:
movies_final.count()

genres          90340
movie_id        90340
budget          90340
title           90331
popularity      90331
release_date    90305
revenue         90331
ROI             26379
Director        90340
dtype: int64

In [32]:
movies_final.to_csv('movies_final_.csv', index=False)